In [1]:
import lammps
import pymc3 as pm
import arviz as az
import numpy as np
import glob
from copy import deepcopy
import matplotlib.pyplot as plt

from lammps_data import LammpsData
from pymatgen.io.vasp import Poscar

In [2]:
def structs_from_poscars(params):
    """
    Reads in POSCARs, creates equivalent lammps inputs.
    
    Args:
        params (dict(dict)): contains core_shell (dict (bool)), charges (dict), and masses (dict).
                             charges contains the charges for each atom type. Key = Atom label(str),
                             value = charge(float)/sub_dict(dict). If atom is core-shell a sub dictionary
                             will be the value, where sub_key = 'core' or 'shell' (float) and sub_value
                             the charge (float). Same format/types uses for masses as in charges.

    Returns:
        structures (list(str)): A list of lammps imput file names generated from the POSCARs.
        structures_data (list(obj)): A list of lammps objects generated from the POSCARs.
    """
    structures = []
    lammps_data = []
    for i, pos in enumerate(glob.glob('poscars/POSCAR*')):
        poscar = Poscar.from_file(pos)
        struct_data = LammpsData.from_structure(poscar.structure, params)
        lammps_data.append(struct_data)
        lammps_file = 'lammps/coords{}.lmp'.format(i+1)
        with open( lammps_file, 'w' ) as f:
            f.write(struct_data.input_string())  
        structures.append(lammps_file)
        
    return structures, lammps_data


In [3]:
def init(lammps_data, structure, cs_springs=None):
    """
    Initialises the system from the structure (read in from lammps input) and non-changing parameters.
    
    Args:
        structure (str): Name of lammps input file.
        cs_springs (optional:dict): A dictionary of the bond_coeff values. Key = atom label and values = K
                                    (energy/distance^2) and r0 (equilibrium bond distance) (list).
                                    Default = None.
        lammps_data (obj): LammpsData stuctural object containing information on atom_types, bond_types,
                           atoms, bonds, cell_lengths, and tilt_factors.

    Returns:
        lmp (Lammps): Lammps system with structure and specified commands implemented.
    """
    
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('read_data {}'.format(structure))

    lmp.command('group cores type {}'.format(lammps_data.type_core()))
    lmp.command('group shells type {}'.format(lammps_data.type_shell()))

    if cs_springs:
        lmp.command('pair_style buck/coul/long/cs 10.0')
        lmp.command('pair_coeff * * 0 1 0')

        lmp.command('bond_style harmonic')
        for i, spring in enumerate(cs_springs):
            lmp.command('bond_coeff {} {} {}'.format(i+1,
                                                     cs_springs[spring][0],
                                                     cs_springs[spring][1]))
    else:
        lmp.command('pair_style buck/coul/long 10.0')
        lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('kspace_style ewald 1e-6')

    #setup for minimization
    lmp.command('min_style cg')

    return lmp

In [4]:
def update_potentials(**kwargs):
    """
    Unpdates the potentials set by pymc3 into the dictionary for the fitting process.

    Args:
        **kwargs: The parameters to be updated in the fitting process as set with pm.Model.

    Returns:
        None.
    """
    for key, value in kwargs.items():    
        for pot in lammps_data[0].potentials:
            if key is pot.a.label_string:
                pot.a.value = value
            if key is pot.rho.label_string:
                pot.rho.value = value
            if key is pot.c.label_string:
                pot.c.value = value
    
def set_potentials(instance):
    """
    Sets the potential for the sepecified Lammps system (changes for each iteration of the potential fit).

    Args:
        instance (obj): Lammps object with structure and specified commands implemented.

    Returns:
        None
    """
    for pot in lammps_data[0].potentials:
        instance.command('{}'.format(pot.potential_string()))

In [5]:
def simfunc(**kwargs):
    """
    Runs a minimization and run for each instance and returns the forces.

    Args:
        **kwargs: Contain data for type of fitting and to what parameters as set with pm.Model.

    Returns:
        out (np.array): x,y,z forces on each atom associated with each given instance.
    """
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([sum(core_mask), 3, len(instances)])
        
        for instance in instances:
            set_potentials(instance)
            instance.command('fix 1 cores setforce 0.0 0.0 0.0')
            instance.command('minimize 1e-25 1e-25 5000 10000')
            instance.command('unfix 1')
            instance.run(0)
            out[:,:,instances.index(instance)] = instance.system.forces[core_mask]
            
    else: out = np.ones([sum(core_mask),3, len(instances)])*999999999 # ThisAlgorithmBecomingSkynetCost
    
    return out

In [6]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  +0.960,
                           'shell': -2.960}}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 15.0,
                          'shell': 1.0} }

params['bpp'] = {'Li-O' : [632.1018, 0.2906, 0.0],
                 'Ni-O' : [1582.5000, 0.2882, 0.0],
                 'O-O'  : [22764.3000, 0.1490, 21.7]}

params['sd'] = {'Li-O' : [50, 0.05, 0.01],
                'Ni-O' : [100, 0.05, 0.05],
                'O-O'  : [100, 0.01, 5]}

#Must be in same order given in charges dictionary
cs_springs = {'O' : [65.0, 0.0]} #Set to None if not using any core-shells

bpp_def = deepcopy(params['bpp'])

structures, lammps_data = structs_from_poscars(params)

Found elements: ['Li', 'Ni', 'O']


In [7]:
instances = [init(lammps_data[i], structures[i], cs_springs) for i,structure in enumerate(structures)]

#Core_mask won't currently work with structure of varying length 
core_mask = lammps_data[0].core_mask()

In [8]:
excude_from_fit = ['Li_O_c','Ni_O_c']

In [9]:
expected = np.zeros([sum(core_mask), 3, len(instances)])

In [10]:
with pm.Model() as model:
#  pm.TruncatedNormal -- truncated so never tries a negative    pm.Uniform
    my_dict = {}
    for pot in lammps_data[0].potentials:
        name = '{}'.format(pot.a.label_string)
        if name not in excude_from_fit:
            my_dict[name] = pot.a.distribution()
        name = '{}'.format(pot.rho.label_string)
        if name not in excude_from_fit:
            my_dict[name] = pot.rho.distribution()
        name = '{}'.format(pot.c.label_string)
        if name not in excude_from_fit:
            my_dict[name] = pot.c.distribution()
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=0.1), draws=1000)
#     trace = pm.sample(step=pm.SMC(ABC=True, epsilon=1000, dist_func="sum_of_squared_distance"), draws=1000)

Sample initial stage: ...
/Users/lmm82/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pymc3/step_methods/smc.py:176: UserWarning: Warning: SMC-ABC methods are experimental step methods and not yet recommended for use in PyMC3!
  warnings.warn(EXPERIMENTAL_WARNING)
Stage: 0 Beta: 0.000 Steps: 25


LinAlgError: 1-th leading minor of the array is not positive definite

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)
plt.savefig('coreshell_LiNiO2_trace3.png',dpi=500, bbox_inches = "tight")

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')
plt.savefig('coreshell_LiNiO2_mode3.png',dpi=500, bbox_inches = "tight")

In [ ]:
# fig, axes = az.plot_forest(trace,
#                            kind='ridgeplot',
#                            var_names=['Li_O_rho','Ni_O_rho', 'O_O_rho'],
#                            combined=True,
#                            ridgeplot_overlap=10,
#                            colors='white',
#                            figsize=(9, 7))

In [ ]:
# pm.summary(trace)

In [ ]:
# def setup_parameters():
#     """
#     Contains and returns the setup information.
    
#     Args:
#         None.

#     Returns:
#         core_shell (dict): A dictionary of booleans defining which elements are core-shell species.
#                            Key = atom label (str) and value = True or False (bool)
#         charges (dict): A dictionary of charges for each atom type. Key = Atom label(str),
#                         value = either the charge (float) for non-core-shell species, or a sub-dictionary
#                         containing { 'core': core_charge (float), 'shell': shell_charge (float) }.
#         structures (list): A list of lammps input file names generated from the POSCARs.
#         elements (list): A list of the elements in the structure, with core and shell entered separately.
#         pairs (list): A list of interacting atom pairs used for defining the pair_coeff.
#         cs_springs (dict): A dictionary of the bond_coeff values. Key = atom label and
#                            values = K (energy/distance^2) and r0 (equilibrium bond distance) (list).
#         bpp_def (dict): A dictionary of the default/starting values for the buckingham potentials.
#                         Key = atom1_atom2_parameter (str) and value = parameter (float).
#         bpp (dict): Same as bpp_def however the values will change during the fitting process.
#                     Key = atom1_atom2_parameter (str) and value = parameter (float).
#     """
    
#     return  structures, structures_data, params, pairs, cs_springs, bpp_def, bpp

# structures, structures_data, params, pairs, cs_springs, bpp_def, bpp = setup_parameters()